In [2]:
import warnings

import pandas as pd 
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from test import *

In [3]:
warnings.filterwarnings('ignore')

In [5]:
train_file=r'loan_data_train.csv'
test_file=r'loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               

In [6]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [23]:
ld_train['State'].dtype=='O'

True

In [7]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
1321,32517.0,5600,5600,6.62%,36 months,wedding,7.23%,NC,MORTGAGE,5833.33,750-754,8,8125,1.0,2 years
295,2164.0,10000,6301.07,14.11%,36 months,debt_consolidation,16.52%,CO,OTHER,4583.00,680-684,16,18707,1.0,3 years
363,40856.0,4000,3975,7.90%,60 months,car,4.48%,TX,OWN,4000.00,800-804,16,6696,0.0,< 1 year
2178,18049.0,9700,9700,14.09%,36 months,debt_consolidation,9.52%,CA,RENT,7333.33,665-669,9,14723,0.0,3 years
980,41484.0,10000,10000,11.71%,36 months,credit_card,12.03%,NC,MORTGAGE,4000.00,695-699,11,8413,0.0,10+ years
1357,71249.0,.,12000,10.16%,36 months,debt_consolidation,27.48%,VA,MORTGAGE,5000.00,710-714,12,10490,0.0,7 years
1177,44737.0,15000,14975,7.90%,36 months,car,20.73%,NY,RENT,5416.67,745-749,14,9756,0.0,1 year
1506,24980.0,10000,10000,13.99%,36 months,debt_consolidation,18.14%,CA,RENT,4166.67,680-684,9,10545,1.0,3 years
498,43056.0,15000,15000,7.90%,36 months,credit_card,22.24%,CA,RENT,2666.67,735-739,15,11105,0.0,< 1 year
265,29283.0,12000,12000,11.49%,36 months,debt_consolidation,12.85%,IL,RENT,2708.33,705-709,8,12150,1.0,< 1 year


In [ ]:


# dont need : ID , Intereste.Rate

# Amount requested : V1
# Amount.Funded.By.Investors : V2 
# Open.CREDIT.Lines : V3
# Revolving.CREDIT.Balance : V4

# convert it to numeric , and then impute missing values with median

# Debt.To.Income.Ratio : V5

# remove percentage sign , covert to numeric and then impute with median

# Loan.Length ,Loan.Purpose,State,Home.Ownership,Employment.Length
# V6-V10
# create dummies for these, considering frequency cutoff as 20

# Monthly.Income : V11,
# Inquiries.in.the.Last.6.Months :V12

# impute missing values with median

# FICO.Range : V13

# split the column at '-', convert the resulting columns to numeric(say a ,b) , 
# then create new column = (a+b)/2 


In [61]:
x = ld_train['State']
data_cols = x
# for col in data_cols:
k=x.value_counts()
print((k<=20).sum())
# print(x.unique())
# print(k.index[k>20])
# print((k==20).sum())

20


In [62]:
ld_train.keys()

Index(['ID', 'Amount.Requested', 'Amount.Funded.By.Investors', 'Interest.Rate',
       'Loan.Length', 'Loan.Purpose', 'Debt.To.Income.Ratio', 'State',
       'Home.Ownership', 'Monthly.Income', 'FICO.Range', 'Open.CREDIT.Lines',
       'Revolving.CREDIT.Balance', 'Inquiries.in.the.Last.6.Months',
       'Employment.Length'],
      dtype='object')

In [ ]:
# pipe_name=pdPipeline([
#     ('name of the process',call to process),
#     ('name of the process', call to process),
#     .....
# ])

In [75]:
p1=pdPipeline([
    ('var_select',Var_Selector(['Amount.Requested','Amount.Funded.By.Investors',
                               'Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',Convert_to_Numeric()),
    ('missing_trt',Impute_Missing())
])



In [11]:
p2=pdPipeline([
    ('var_select',Var_Selector(['Debt.To.Income.Ratio'])),
    ('string_clean',String_Replacement(replace_it='%',replace_with='')),
    ('convert_to_numeric',Convert_to_Numeric()),
    ('missing_trt',Impute_Missing())
])

In [12]:
p3=pdPipeline([
    ('var_select',Var_Selector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership',
                               'Employment.Length'])),
    ('missing_trt',Impute_Missing()),
    ('create_dummies',Categorigal_Dummies(20))
])

In [13]:
p4=pdPipeline([
    ('var_select',Var_Selector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',Impute_Missing())
])

In [14]:
p5=pdPipeline([
    ('var_select',Var_Selector(['FICO.Range'])),
    ('custom_fico',Fico_separation()),
    ('missing_trt',Impute_Missing())
])

In [76]:
data_pipe=FeatureUnion([
    ('obj_to_num',p1)
    # ('dtir',p2),
    # ('obj_to_dum',p3),
    # ('num',p4),
    # ('fico',p5)
])

In [77]:
data_pipe.fit(ld_train)

TypeError: data type '' not understood

In [34]:
len(data_pipe.get_feature_names())

4

In [ ]:
data_pipe.transform(ld_train).shape

In [ ]:
x_train=pd.DataFrame(data=data_pipe.transform(ld_train),
                    columns=data_pipe.get_feature_names())

In [ ]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                    columns=data_pipe.get_feature_names())

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train.columns

In [ ]:
x_test.columns